# Generate Figure of Merit based dataset

This notebook is to generate a set of DR2-lite style `.par` and `.tim` files based on a figure of merit (FoM).
We include functions for a burst with memory (BWM) and stochastic gravitational wave background (gwb) FoM.

In [1]:
from __future__ import division, print_function, unicode_literals

import numpy as np
import os, glob
import json
import matplotlib.pyplot as plt
import libstempo as t2
from enterprise.signals import utils

import dr2lite_utils as dr2u

%matplotlib inline

Update paths to reflect your usage.

In [2]:
DR2dir = os.path.abspath('data_release/IPTA-DR2/')
datadir = os.path.join(DR2dir, 'release/VersionB')

lite_dir = os.path.join(DR2dir, 'DR2lite_partim')  # cleaned/combined par/tim files
os.system('mkdir -p {}'.format(lite_dir));

# 1) clean up `.par` and `.tim` files

## make list of pulsar names for later

In [3]:
psrlist = []
psrs = glob.glob(datadir + '/J*')
for psr in psrs:
    name = psr.split('/')[-1]
    psrlist.append(name)
psrlist.sort()

## clean `.par` files

In [4]:
parfiles = glob.glob(datadir + '/J*/*IPTADR2.par')

for p in parfiles:
    name = p.split('/')[-2]
    outfile = os.path.join(lite_dir, '{}.par'.format(name))
    dr2u.clean_par(p, outfile)

## combine `.tim` files

In [5]:
timfiles = glob.glob(datadir + '/J*/*IPTADR2.tim')

for t in timfiles:
    name = t.split('/')[-2]
    outfile = os.path.join(lite_dir, '{}.tim'.format(name))
    dr2u.combine_tim(t, outfile)

# 2) Figure of Merit Calculation

Calculate the FoM for each pulsar for each PTA that observes it.
Both of these FoMs are for white noise dominated pulsars, which is not true... You may want to modify them.

## Burst with Memory FoM
From van Haasteren & Levin (2010) the uncertainty on the measurement of the BWM amplitude is estimated for even cadence, white-noise dominated observations.  Madison *et al*. (2014) interpret this as the minimum amplitude for a 1-$\sigma$ detection.

$$ \sigma_{h_0} = h_{min} \sim \sigma_\star \,\Delta t^{1/2}\, T^{-3/2}, $$

where $\sigma_\star$ is the TOA measurment uncertainty for a single pulsar or the response weighted root-mean-squared of uncertainty for all pulsars in a network, $\Delta t$ is the observing cadence, and $T$ the observation baseline.

To maximize the sensitivity of our array we want to minimize $\sigma_{h_0}$.
We define our figure of merit for pulsar selection as:
$$ \mathrm{FoM}_{bwm} = \frac{1}{\sigma_{h_0}} = \sqrt{\frac{T^{3}}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t}} $$

## Gravitational Wave Background FoM

From GWB signal-to-noise ratio (SNR) scaling laws in Siemens, et al. 2013. 

The weak signal scaling law, used at the December 2017 busyweek at Flatiron CCA in NYC gives a FoM of:

$$ \mathrm{FoM}_{gwb} = \frac{T^\frac{13}{3}}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t} $$

And the intermediate regime sclaing gives the following Figure of Merit:

$$ \mathrm{FoM}_{gwb} = T^\frac{1}{2}\left(\frac{1}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t}\right)^\frac{1}{2\beta} $$

In [6]:
def bwm_FoM(Tobs, dt, TOAerr):
    """BWM FoM = 1/sig_h0
    """
    return np.sqrt(Tobs**3/dt) / TOAerr

def gwb_FoM_weak(Tobs, dt, TOAerr):
    """GWB FoM from SNR scaling laws
    """
    return Tobs**(13/3) / TOAerr**2 / dt 

def gwb_FoM_inter(Tobs, dt, TOAerr):
    """GWB FoM from SNR scaling laws
    """
    return Tobs**(1/2) * (TOAerr**2 * dt) ** (-1/(2*13/3)) 


In [7]:
my_FoM = gwb_FoM_inter # function alias for FoM of choice

In [8]:
# analysis tag to use in filenames
tag = "gwb_7yr_BW1p1_DM_10_sigmax_10_NEW"

# filter parameters
BW = 1.1
DM_window = 10

# best PSR selection parameters
Tmin = 7  # yrs; if >9 then no NANOGrav pulsars selected since only 9-yr data present.
sigmax = 10.0  # us

In [9]:
ptas = ['PPTA', 'EPTA', 'NANOGrav']
backends = {'NANOGrav': 
                ['327_ASP', '430_ASP', 'L-wide_ASP', 'S-wide_ASP',
                 '327_PUPPI', '430_PUPPI', 'L-wide_PUPPI',  'S-wide_PUPPI',
                 'Rcvr_800_GASP', 'Rcvr1_2_GASP',
                 'Rcvr_800_GUPPI', 'Rcvr1_2_GUPPI',
                ],
            'PPTA': 
                ['PDFB_10CM', 'PDFB_20CM', 'PDFB_40CM',
                 'CPSR2_20CM', 'CPSR2_50CM',
                 'WBCORR_10CM', 'WBCORR_20CM',
                ],
                
            'EPTA': 
                ['EFF.EBPP.1360', 'EFF.EBPP.1410', 'EFF.EBPP.2639',
                 'JBO.DFB.1400', 'JBO.DFB.1520',
                 'NRT.BON.1400', 'NRT.BON.1600', 'NRT.BON.2000',
                 'WSRT.P1.328', 'WSRT.P1.328.C', 'WSRT.P1.323.C',
                 'WSRT.P1.382', 'WSRT.P1.382.C', 'WSRT.P1.367.C',
                 'WSRT.P1.840', 'WSRT.P1.840.C',
                 'WSRT.P1.1380', 'WSRT.P1.1380.C',
                 'WSRT.P1.1380.1',
                 'WSRT.P1.1380.2', 'WSRT.P1.1380.2.C',
                 'WSRT.P1.2273.C',
                ]
           }  # backends to use for each PTA, values for `-group` flag

### Backend Notes:
#### NANOGrav

legacy, not used (`-group` flag)
```
kaspi
kaspi23 (-be mk3_23m, mk2_23)
kaspi14 (-be mk3_14m, mk2_14, mk3_14r)
L-band_Mark3a
L-band_Mark3b
L-band_Mark4
S-band_Mark4
L-band_ABPP
S-band_ABPP
```

#### PPTA

backend list for `-f` flag
```
['1050CM_PDFB1', '1050CM_PDFB2', '1050CM_PDFB3', '1050CM_PDFB4',
 '1050CM_WBCORR',
 'H-OH_CPSR2m', 'H-OH_CPSR2n', 'H-OH_PDFB1',
 'MULTI_CPSR2m', 'MULTI_CPSR2n',
 'MULTI_PDFB1', 'MULTI_PDFB2', 'MULTI_PDFB3', 'MULTI_PDFB4',
 'MULTI_WBCORR',
]
```
small number of TOAs
```
CPSR2_40CM -- J0437 & J0711 & J1045 & J2129 TOA total <3 TOAs / psr
PDFB1_early_10CM -- J1824-2452A -- 7 TOAs total
PDFB1_early_20CM -- J1824-2452A -- 1 TOA
PDFB_50CM -- J0437 & J1045, 2 TOAs total
PDFB1_20CM -- J1824-2452A, 4 TOAs total
PDFB1_10CM -- J1824-2452A, 13 TOAs total
WBCORR_20CM -- 11 Parkes PSRs, 30 TOAs for J0437, rest with <10 (still used... should it be?)
```
there are some backends with `legacy` in the name... I'm not using those.  They have lots of missing metadata.
```
PKS.fptm.20cm-legacy
PKS.s2.20cm-legacy
PKS.cpsr1.20cm_legacy
PKS.cpsr2.20cm_legacy
PKS.AFB.20cm_legacy
PKS.cpsr2.50cm_legacy
```

#### EPTA

possible legacy, only used for single freq J1939
```
NRT.DDS.1400
```
small number of TOAs
```
JBO.DFB.5000 -- J1713, 4 TOAs

```

### Construct `FoM_dict` (takes a long time, skip if already saved)

Runs `filter_psr()` for each pulsar for each PTA, computing the FoM.

In [10]:
redo = False

dict_file = 'FoM_output/FoM_{}_dict.json'.format(tag)  # where to save FoM dictionary
data_file = 'FoM_output/FoM_{}.dat'.format(tag)  # plain text file of FoM data

In [11]:
if redo or not os.path.isfile(dict_file):
    FoM_dict = {}
    fout = open(data_file, 'w')
    fout.write('{:11}{:9}{:9}{:9}{:9}{:9}{:12}\n'
               .format(' Pulsar', ' PTA', ' raw_Tobs', '  mf_Tobs', '  TOAerr', '  cadence', '      FoM'))

    for psrName in psrlist:
        FoM_dict[psrName] = {}
    
        parfile = os.path.join(lite_dir, '{}.par'.format(psrName))
        timfile = os.path.join(lite_dir, '{}.tim'.format(psrName))
        psr = t2.tempopulsar(parfile, timfile, maxobs=30000)
        for pta in ptas:
            if pta in psr.flagvals('pta'):
                FoM_dict[psrName][pta] = {}
                filt = {'pta':[pta], 'group':backends[pta]}
                psr = dr2u.filter_psr(psr, bw=BW, dt=DM_window, filter_dict=filt, plot=False)
                
                idx = ~psr.deletedmask()
                Tobs_raw = (psr.toas().max()-psr.toas().min()) / 365.25
                if len(psr.toas()[idx]) > 0:
                    U, _ = utils.create_quantization_matrix(psr.toas()[idx]*86400, dt=86400, nmin=1)
                    cadence = (psr.toas()[idx].max() - psr.toas()[idx].min()) / U.shape[1] # mean dt
                    Tobs = (psr.toas()[idx].max()-psr.toas()[idx].min()) / 365.25
                    sigma = 1 / np.mean(1/psr.toaerrs[idx])  # harmonic mean TOAerr
                    FoM = my_FoM(Tobs, cadence, sigma)
                else:
                    Tobs = 0
                    sigma = np.inf
                    cadence = np.inf
                    FoM = 0
                fout.write('{:11}{:9}{:9.2f}{:9.2f}{:9.2f}{:9.2f}{:9.2f}\n'
                            .format(psrName, pta, Tobs_raw, Tobs, sigma, cadence, FoM))
                FoM_dict[psrName][pta]['Tobs'] = Tobs
                FoM_dict[psrName][pta]['dt'] = cadence
                FoM_dict[psrName][pta]['sigma'] = sigma
                FoM_dict[psrName][pta]['FoM'] = FoM
        del psr
    fout.close()

Working on PSR J0023+0923
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP', u'Rcvr1_2_GASP', u'Rcvr_800_GUPPI', u'Rcvr1_2_GUPPI']
Keeping TOAs corresponding to pta [u'NANOGrav']
Running multi-frequency filter
Cutting 0 TOAs
Working on PSR J0030+0451
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
backends marked as 'orphan': [['EFF.EBPP.1410', 4]]
Cutting 2980 TOAs
Turning off fit for ['JUMP2', 'JUMP9']
All TOAs are bein

backends marked as 'orphan': [['JBO.DFB.1400', 4]]
Cutting 464 TOAs
Turning off fit for ['JUMP4']
Working on PSR J0931-1902
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP', u'Rcvr1_2_GASP', u'Rcvr_800_GUPPI', u'Rcvr1_2_GUPPI']
Keeping TOAs corresponding to pta [u'NANOGrav']
Running multi-frequency filter
Cutting 0 TOAs
Working on PSR J1012+5307
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
backends marked as 'orphan':

Cutting 0 TOAs
Working on PSR J1640+2224
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
backends marked as 'orphan': [['EFF.EBPP.1410', 7], ['JBO.DFB.1400', 5], ['NRT.BON.1600', 6], ['WSRT.P1.1380.C', 9]]
Cutting 2884 TOAs
Turning off fit for ['JUMP3', 'JUMP5', 'JUMP7', 'JUMP8', 'JUMP10']
All TOAs are being jumped!
Setting JUMP1 as reference jump.
Working on PSR J1640+2224
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP

Cutting 288 TOAs
Working on PSR J1744-1134
Keeping TOAs corresponding to group [u'PDFB_10CM', u'PDFB_20CM', u'PDFB_40CM', u'CPSR2_20CM', u'CPSR2_50CM', u'WBCORR_10CM', u'WBCORR_20CM']
Keeping TOAs corresponding to pta [u'PPTA']
Running multi-frequency filter
backends marked as 'orphan': [['WBCORR_20CM', 1]]
Cutting 9472 TOAs
Turning off fit for ['JUMP1', 'JUMP2', 'JUMP3', 'JUMP4', 'JUMP5', 'JUMP6', 'JUMP7', 'JUMP8', 'JUMP9', 'JUMP10', 'JUMP24', 'JUMP26', 'JUMP27', 'JUMP28', 'JUMP29', 'JUMP30', 'JUMP31', 'JUMP32', 'JUMP33', 'JUMP34']
All TOAs are being jumped!
Setting JUMP25 as reference jump.
Working on PSR J1744-1134
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSR

Cutting 999 TOAs
Working on PSR J1903+0327
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP', u'Rcvr1_2_GASP', u'Rcvr_800_GUPPI', u'Rcvr1_2_GUPPI']
Keeping TOAs corresponding to pta [u'NANOGrav']
Running multi-frequency filter
Cutting 10 TOAs
Working on PSR J1909-3744
Keeping TOAs corresponding to group [u'PDFB_10CM', u'PDFB_20CM', u'PDFB_40CM', u'CPSR2_20CM', u'CPSR2_50CM', u'WBCORR_10CM', u'WBCORR_20CM']
Keeping TOAs corresponding to pta [u'PPTA']
Running multi-frequency filter
backends marked as 'orphan': [['WBCORR_20CM', 2]]
Cutting 10780 TOAs
Turning off fit for ['JUMP1', 'JUMP2', 'JUMP3', 'JUMP5', 'JUMP13']
All TOAs are being jumped!
Setting JUMP8 as reference jump.
Working on PSR J1909-3744
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000

No multi-frequency data, returning original psr
Working on PSR J1955+2908
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP', u'Rcvr1_2_GASP', u'Rcvr_800_GUPPI', u'Rcvr1_2_GUPPI']
Keeping TOAs corresponding to pta [u'NANOGrav']
Running multi-frequency filter
Cutting 393 TOAs
Turning off fit for ['JUMP1', 'JUMP2', 'JUMP3', 'JUMP4']
Working on PSR J2010-1323
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
backends marked as 

Cutting 0 TOAs
Working on PSR J2317+1439
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
backends marked as 'orphan': [['EFF.EBPP.1410', 2], ['JBO.DFB.1400', 3], ['WSRT.P1.1380.C', 8]]
Cutting 3027 TOAs
Turning off fit for ['JUMP3', 'JUMP5', 'JUMP8', 'JUMP10']
All TOAs are being jumped!
Setting JUMP1 as reference jump.
Working on PSR J2317+1439
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP', u'Rcvr1_2_GASP', u'Rcvr_800

### save `FoM_dict` as `json` database for later usage

In [12]:
if redo or not os.path.isfile(dict_file):

    # convert FoM_dict to python float (json can't handle numpy.float128)
    for psr in FoM_dict.keys():
        for pta in FoM_dict[psr].keys():
            for key in FoM_dict[psr][pta].keys():
                FoM_dict[psr][pta][key] = float(FoM_dict[psr][pta][key])

    with open(dict_file, 'w') as fout:
        json.dump(FoM_dict, fout, sort_keys=True, indent=4, separators=(',', ': '))

# 3) Select pulsars to use

Use only PSRs with `Tmin` yrs of multi-frequency data.  Pick one PTA per pulsar.

In [13]:
#dict_file = 'FoM_output/FoM_{}_dict.json'.format(tag)  # FoM dictionary, same as above
with open(dict_file, 'r') as fin:
    FoM_dict = json.load(fin)

In [14]:
psrdict = {}

for psrname in psrlist:
    best = {'pta':None, 'Tobs':0, 'FoM':0}
    for pta in ptas:
        try:
            this = FoM_dict[psrname][pta]
            if this['Tobs'] > Tmin and this['sigma'] < sigmax and this['FoM'] > best['FoM']:
                best = {'pta':pta, 'Tobs':this['Tobs'], 'FoM':this['FoM']}
        except KeyError:
            pass
    if best['pta'] is not None:
        psrdict.update({psrname: {'pta':[best['pta']], 'group':backends[best['pta']]}})
        print('{}: {pta:8}, T = {Tobs:5.2f} yr, FoM = {FoM:6.2f}'.format(psrname, **best))

J0030+0451: NANOGrav, T =  8.77 yr, FoM =   1.73
J0034-0534: EPTA    , T =  9.87 yr, FoM =   1.47
J0218+4232: EPTA    , T = 13.54 yr, FoM =   1.73
J0437-4715: PPTA    , T = 10.78 yr, FoM =   4.37
J0613-0200: EPTA    , T = 13.67 yr, FoM =   2.31
J0621+1002: EPTA    , T =  7.50 yr, FoM =   1.19
J1012+5307: EPTA    , T = 13.13 yr, FoM =   2.27
J1022+1001: EPTA    , T =  7.35 yr, FoM =   1.62
J1024-0719: EPTA    , T =  7.49 yr, FoM =   1.58
J1455-3330: NANOGrav, T =  9.22 yr, FoM =   1.47
J1600-3053: EPTA    , T =  7.39 yr, FoM =   2.27
J1640+2224: NANOGrav, T =  8.89 yr, FoM =   2.13
J1643-1224: EPTA    , T =  7.07 yr, FoM =   1.81
J1713+0747: EPTA    , T = 13.91 yr, FoM =   3.94
J1744-1134: PPTA    , T = 10.69 yr, FoM =   2.91
J1857+0943: NANOGrav, T =  8.87 yr, FoM =   2.08
J1909-3744: PPTA    , T = 10.82 yr, FoM =   4.12
J1918-0642: NANOGrav, T =  9.02 yr, FoM =   1.73
J1939+2134: EPTA    , T = 12.51 yr, FoM =   4.90
J2145-0750: EPTA    , T =  7.06 yr, FoM =   1.93
J2317+1439: NANOGrav

In [15]:
E,N,P = 0,0,0
for val in psrdict.values():
    val = val['pta']
    if 'EPTA' in val:
        E += 1
    elif 'NANOGrav' in val:
        N += 1
    elif 'PPTA' in val:
        P += 1
print(' EPTA - {:2d}\n   NG - {:2d}\n PPTA - {:2d}'.format(E,N,P))

 EPTA - 12
   NG -  6
 PPTA -  3


In [16]:
# save `psrdict` to file
fname = 'psrdicts/FoM_{}_psrs.json'.format(tag)
with open(fname, 'w') as fout:
    json.dump(psrdict, fout, sort_keys=True, indent=4, separators=(',', ': '))

# 3) Create filtered par and tim files for DR2-lite analysis

In [17]:
# directory where you want the dataset saved... change for your usage
rundir = 'FoM_output/FoM_{}'.format(tag)

dr2u.make_dataset(psrdict, indir=lite_dir, outdir=rundir,
                  bw=BW, dt=DM_window, plot=False)

Working on PSR J0030+0451
Keeping TOAs corresponding to group [u'327_ASP', u'430_ASP', u'L-wide_ASP', u'S-wide_ASP', u'327_PUPPI', u'430_PUPPI', u'L-wide_PUPPI', u'S-wide_PUPPI', u'Rcvr_800_GASP', u'Rcvr1_2_GASP', u'Rcvr_800_GUPPI', u'Rcvr1_2_GUPPI']
Keeping TOAs corresponding to pta [u'NANOGrav']
Running multi-frequency filter
Cutting 912 TOAs
Turning off fit for ['JUMP1', 'JUMP2', 'JUMP3', 'JUMP4', 'JUMP5', 'JUMP6', 'JUMP7']
filtered Tobs = 8.77 yrs


Working on PSR J0034-0534
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
Cutting 72 TO

All TOAs are being jumped!
Setting JUMP1 as reference jump.
filtered Tobs = 7.07 yrs


Working on PSR J1713+0747
Keeping TOAs corresponding to group [u'EFF.EBPP.1360', u'EFF.EBPP.1410', u'EFF.EBPP.2639', u'JBO.DFB.1400', u'JBO.DFB.1520', u'NRT.BON.1400', u'NRT.BON.1600', u'NRT.BON.2000', u'WSRT.P1.328', u'WSRT.P1.328.C', u'WSRT.P1.323.C', u'WSRT.P1.382', u'WSRT.P1.382.C', u'WSRT.P1.367.C', u'WSRT.P1.840', u'WSRT.P1.840.C', u'WSRT.P1.1380', u'WSRT.P1.1380.C', u'WSRT.P1.1380.1', u'WSRT.P1.1380.2', u'WSRT.P1.1380.2.C', u'WSRT.P1.2273.C']
Keeping TOAs corresponding to pta [u'EPTA']
Running multi-frequency filter
Cutting 16762 TOAs
Turning off fit for ['JUMP6', 'JUMP16', 'JUMP17', 'JUMP18', 'JUMP19', 'JUMP20', 'JUMP21', 'JUMP23', 'JUMP24', 'JUMP25', 'JUMP26', 'JUMP27', 'JUMP28', 'JUMP29', 'JUMP30', 'JUMP31', 'JUMP32', 'JUMP33', 'JUMP34', 'JUMP35', 'JUMP36', 'JUMP37', 'JUMP38', 'JUMP39', 'JUMP40', 'JUMP41', 'JUMP42', 'JUMP43', 'JUMP44', 'JUMP45', 'JUMP46', 'JUMP47', 'JUMP48', 'JUMP49', 'JUMP